In [1]:
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary
from pyquil.quil import *

In [2]:
import numpy as np
import math
from math import pi
import random
import copy
from tqdm import tqdm_notebook as tqdm

In [3]:
from functions import *

In [4]:
# if __name__ == "__main__":
#     target_qubit = [0]
#     num_qubits = len(target_qubit)

# #     First step choose m and the K_m sequences of 
#     m = 1
#     k_m = 1 #n. of diff sequences
#     n_m = 1  #n. of samples from a certain sequence

In [9]:
def native_universal_two_qubits_packs_generator(qmachine, target_qubits:list, num_layer:int):
    list_gates = []
    for index in range(num_layer):
        draft_circuit = give_random_two_qubit_circuit(target_qubits)
        list_gates.extend( qmachine.compiler.quil_to_native_quil(draft_circuit) )
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    return list_gates

def native_rigetti_single_qubit_packs_generator(qmachine, target_qubit, num_layer:int):
    try:
        temp = iter(target_qubit)
        if len(target_qubit) == 1:
            target_qubit = target_qubit[0]
        else:
            raise ValueError('target qubit should be only one index')
    except:
        pass
    
    list_gates = []
    angles = np.linspace(0, np.pi, 100)
    
    for index in range(0,num_layer):
        omega, phi = np.random.uniform(0, 2*np.pi, size = 2)
        theta = np.random.choice(angles, p = np.sin(angles) / np.sum( np.sin(angles) ))
        
        draft_circuit = Program( [RZ(phi, qubit = target_qubit),
                                  RY(theta, qubit = target_qubit),
                                  RZ(omega, qubit = target_qubit)])
        
        list_gates.extend(qmachine.compiler.quil_to_native_quil(draft_circuit))
    
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    return list_gates

In [56]:
def used_qubits_index(gates_sequence):
    qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])
    qubits = np.array([ ele.index for sub_arr in qubits for ele in sub_arr]) #some gates might have multiple indices
    qubits_indices = np.unique(qubits)
    qubits_indices.sort()
    return qubits_indices

def run_bench_experiment(qmachine, program):
    #Run the program
    executable = qmachine.compile(program)
    result = qmachine.run(executable)
    measured_outcome = result.readout_data.get('ro')
    return 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state

def get_inverse_circuit(gates_sequence):
    """
    :params gates_sequence: iterable sequence of circuit gates.
    :return: numpy array of gates constructing inverse circuit of the input 
    """
    target_qubits = used_qubits_index(gates_sequence)
    n_qubits = len(target_qubits)
    
    prog = Program(gates_sequence)
    u_inverse_definition = DefGate('U_inverse', np.linalg.inv(program_unitary(prog, n_qubits=n_qubits)))
    U_inverse = u_inverse_definition.get_constructor()
    prog += qmachine.compiler.quil_to_native_quil(Program(u_inverse_definition, U_inverse(*target_qubits)))

    #Do not let the quilc to alter the gates by optimization
    prog = Program('PRAGMA PRESERVE_BLOCK') + prog
    prog += Program('PRAGMA END_PRESERVE_BLOCK')
    
    instructions = prog.instructions
    inverting_gates_list = [ ins for ins in instructions if isinstance(ins, Gate)]
    return np.array(inverting_gates_list)

def generate_experiments(qmachine, target_qubits:list, circuit_gen_func, layers_num:int, exp_num:int):
    n_qubits = len(target_qubits)
    return np.array([circuit_gen_func(qmachine, target_qubits, layers_num) for i in range(exp_num)])

def bench_machine(qmachine, rb_experiments, number_of_shots):
    """
    It samples and record the accept or reject of the machine with given gate sequences
    :return: response_matrix including accepts and rejects in columns
    """
    target_qubits = used_qubits_index(rb_experiments[0])
    n_qubits = len(target_qubits)
    sequ_num = len(rb_experiments)
    response_matrix = np.zeros((sequ_num, number_of_shots))

    for i_sequ, sequ in enumerate(rb_experiments):
        prog = Program() #All qubits begin with |0> state
        for gate in sequ:
            prog += gate
        
        #Measurments
        ro = prog.declare('ro', 'BIT', n_qubits)
        for ind, qubit_ind in enumerate(target_qubits):
            prog += MEASURE(qubit_ind, ro[ind])
        prog = prog.wrap_in_numshots_loop(number_of_shots)

        response_matrix[i_sequ,:] = run_bench_experiment(qmachine, prog)
    return response_matrix

In [43]:
qc = get_qc("9q-square-noisy-qvm")

In [30]:
exps = generate_experiments(qmachine = qc, target_qubits = [0],
                           circuit_gen_func=native_rigetti_single_qubit_packs_generator, layers_num=2, exp_num=2)
response_matrix = bench_machine(qc, exps, 100)
response_matrix

array([[1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
        0., 0., 0., 1.],
       [1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1.]])

In [47]:
exps = generate_experiments(qmachine = qc, target_qubits = [0,1],
                           circuit_gen_func=native_universal_two_qubits_packs_generator, layers_num=2, exp_num=2)
response_matrix = bench_machine(qc, exps, 100)
response_matrix

C:\Users\mohsen\AppData\Local\Temp\ipykernel_20096\932765169.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qubits = np.array([np.array(gate.qubits) for gate in gates_sequence]).flatten()


TypeError: qubit should be an int or Qubit or QubitPlaceholder instance

In [ ]:
Qubit(0).index

In [58]:
used_qubits_index(native_universal_two_qubits_packs_generator(qc,[0,1],2))

C:\Users\mohsen\AppData\Local\Temp\ipykernel_20096\2340585608.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  qubits = np.array([np.array(gate.qubits) for gate in gates_sequence])


array([0, 1])

In [41]:
np.array( CNOT(0,1).qubits ).flatten()

array([<Qubit 0>, <Qubit 1>], dtype=object)